# Assignment 2: Parts-of-Speech Tagging (POS)

Welcome to the second assignment of Course 2 in the Natural Language Processing specialization. This assignment will develop skills in part-of-speech (POS) tagging, the process of assigning a part-of-speech tag (Noun, Verb, Adjective...) to each word in an input text.  Tagging is difficult because some words can represent more than one part of speech at different times. They are  **Ambiguous**. Let's look at the following example: 

- The whole team played **well**. [adverb]
- You are doing **well** for yourself. [adjective]
- **Well**, this assignment took me forever to complete. [interjection]
- The **well** is dry. [noun]
- Tears were beginning to **well** in her eyes. [verb]

Distinguishing the parts-of-speech of a word in a sentence will help us better understand the meaning of a sentence. This would be critically important in search queries. Identifying the proper noun, the organization, the stock symbol, or anything similar would greatly improve everything ranging from speech recognition to search. By completing this assignment, we will: 

- Learn how parts-of-speech tagging works
- Compute the transition matrix A in a Hidden Markov Model
- Compute the transition matrix B in a Hidden Markov Model
- Compute the Viterbi algorithm 
- Compute the accuracy of our model 


## Outline

- [0 Data Sources](#0)
- [1 POS Tagging](#1)
    - [1.1 Training](#1.1)
        - [Exercise 01](#ex-01)
    - [1.2 Testing](#1.2)
        - [Exercise 02](#ex-02)
- [2 Hidden Markov Models](#2)
    - [2.1 Generating Matrices](#2.1)
        - [Exercise 03](#ex-03)
        - [Exercise 04](#ex-04)
- [3 Viterbi Algorithm](#3)
    - [3.1 Initialization](#3.1)
        - [Exercise 05](#ex-05)
    - [3.2 Viterbi Forward](#3.2)
        - [Exercise 06](#ex-06)
    - [3.3 Viterbi Backward](#3.3)
        - [Exercise 07](#ex-07)
- [4 Predicting on a data set](#4)
    - [Exercise 08](#ex-08)

In [3]:
# Importing packages and loading in the data set 
#from utils_pos import get_word_tag, preprocess  
import pandas as pd
from collections import defaultdict
import math
import string
import numpy as np

In [4]:
def assign_unk(word):
    """
    Assign tokens to unknown words
    """
    punctuation = set(string.punctuation)    # Punctuation characters
    
    # Suffixes
    noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
    verb_suffix = ["ate", "ify", "ise", "ize"]
    adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
    adv_suffix = ["ward", "wards", "wise"]

    # Loop the characters in the word, check if any is a digit
    if any(char.isdigit() for char in word):
        return '--unk_digit--'
    
    # Loop the characters in the word, check if any is a punctuation character
    elif any(char in punctuation for char in word):
        return '--unk_punct--'
    
    # Loop the characters in the word, check if any is an upper case character
    elif any(char.isupper() for char in word):
        return '--unk_upper--'
    
    # Check if word ends with any noun suffix
    elif any(word.endswith(ns) for ns in noun_suffix):
        return '--unk_noun--'
    
    # Check if word ends with any verb suffix
    elif any(word.endswith(vb) for vb in verb_suffix):
        return '--unk_verb--'
    
    # Check if word ends with any adjective suffix
    elif any(word.endswith(adj) for adj in adj_suffix):
        return '--unk_adj--'
    
    # Check if word ends with any adverb suffix
    elif any(word.endswith(adv) for adv in adv_suffix):
        return '--unk_adv--'
    
    # If none of the previous criteria is met, return plain unknown
    else:
        return '--unk--'

In [5]:
def get_word_tag(line, vocab):
    # If line is empty return placeholders for word and tag else Split line to separate word and tag
    # Check if word is not in vocabulary
    # Handle unknown word
    words = line.split()
    if not words:
        return ('--n--', '--s--')
    elif words[0] not in vocab:
        return (assign_unk(words[0]), words[1])
    else:
        return tuple(words)

<a name='0'></a>
## Part 0: Data Sources
In this assignment, we will use two tagged data sets collected from the **Wall Street Journal (WSJ)**. 

[Here](http://relearn.be/2015/training-common-sense/sources/software/pattern-2.6-critical-fork/docs/html/mbsp-tags.html) is an example 'tag-set' or Part of Speech designation describing the two or three letter tag and their meaning. 
- One data set (**WSJ-2_21.pos**) will be used for **training**.
- The other (**WSJ-24.pos**) for **testing**. 
- The tagged training data has been preprocessed to form a vocabulary (**hmm_vocab.txt**). 
- The words in the vocabulary are words from the training set that were used two or more times. 
- The vocabulary is augmented with a set of 'unknown word tokens', described below. 

The training set will be used to create the emission, transmission and tag counts. 

The test set (WSJ-24.pos) is read in to create `y`. 
- This contains both the test text and the true tag. 
- The test set has also been preprocessed to remove the tags to form **test_words.txt**. 
- This is read in and further processed to identify the end of sentences and handle words not in the vocabulary using functions provided in **utils_pos.py**. 
- This forms the list `test_corpus`, the preprocessed text used to test our  POS taggers.

A POS tagger will necessarily encounter words that are not in its datasets. 
- To improve accuracy, these words are further analyzed during preprocessing to extract available hints as to their appropriate tag. 
- For example, the suffix 'ize' is a hint that the word is a verb, as in 'final-ize' or 'character-ize'. 
- A set of unknown-tokens, such as '--unk-verb--' or '--unk-noun--' will replace the unknown words in both the training and test corpus and will appear in the emission, transmission and tag data structures.


<img src = "DataSources1.PNG" />

Implementation note: 

- For python 3.6 and beyond, dictionaries retain the insertion order. 
- Furthermore, their hash-based lookup makes them suitable for rapid membership tests. 
    - If _di_ is a dictionary, `key in di` will return `True` if _di_ has a key _key_, else `False`. 

The dictionary `vocab` will utilize these features.

In [6]:
# load in the training corpus in WSJ_02-21.pos
with open('WSJ_02-21.pos', 'r') as f:
    training_corpus = f.readlines()

print(f"A few items of the training corpus list")
print(training_corpus[0:5])

A few items of the training corpus list
['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n']


In [7]:
# read the vocabulary data hmm_vocab.txt, split by each line of text, and save the list
with open('hmm_vocab.txt', 'r') as f:
    voc_l = f.read().split('\n')
    
print("A few items of the vocabulary list")
print(voc_l[0:50])
print()
print("A few items at the end of the vocabulary list")
print(voc_l[-50:])

A few items of the vocabulary list
['!', '#', '$', '%', '&', "'", "''", "'40s", "'60s", "'70s", "'80s", "'86", "'90s", "'N", "'S", "'d", "'em", "'ll", "'m", "'n'", "'re", "'s", "'til", "'ve", '(', ')', ',', '-', '--', '--n--', '--unk--', '--unk_adj--', '--unk_adv--', '--unk_digit--', '--unk_noun--', '--unk_punct--', '--unk_upper--', '--unk_verb--', '.', '...', '0.01', '0.0108', '0.02', '0.03', '0.05', '0.1', '0.10', '0.12', '0.13', '0.15']

A few items at the end of the vocabulary list
['yards', 'yardstick', 'year', 'year-ago', 'year-before', 'year-earlier', 'year-end', 'year-on-year', 'year-round', 'year-to-date', 'year-to-year', 'yearlong', 'yearly', 'years', 'yeast', 'yelled', 'yelling', 'yellow', 'yen', 'yes', 'yesterday', 'yet', 'yield', 'yielded', 'yielding', 'yields', 'you', 'young', 'younger', 'youngest', 'youngsters', 'your', 'yourself', 'youth', 'youthful', 'yuppie', 'yuppies', 'zero', 'zero-coupon', 'zeroing', 'zeros', 'zinc', 'zip', 'zombie', 'zone', 'zones', 'zoning', '{',

In [8]:
# vocab: dictionary that has the index of the corresponding words
vocab = {word: i for i, word in enumerate(voc_l)}

# Get the index of the corresponding words. 
print('Vocabulary dictionary, key is the word, value is a unique integer')
list(vocab.items())[:20]

Vocabulary dictionary, key is the word, value is a unique integer


[('!', 0),
 ('#', 1),
 ('$', 2),
 ('%', 3),
 ('&', 4),
 ("'", 5),
 ("''", 6),
 ("'40s", 7),
 ("'60s", 8),
 ("'70s", 9),
 ("'80s", 10),
 ("'86", 11),
 ("'90s", 12),
 ("'N", 13),
 ("'S", 14),
 ("'d", 15),
 ("'em", 16),
 ("'ll", 17),
 ("'m", 18),
 ("'n'", 19)]

In [9]:
# load in the test corpus WSJ_24.pos
with open('WSJ_24.pos', 'r') as f:
    test_tag_corpus = f.readlines()
    
print(test_tag_corpus[:10])
len(test_tag_corpus)

['The\tDT\n', 'economy\tNN\n', "'s\tPOS\n", 'temperature\tNN\n', 'will\tMD\n', 'be\tVB\n', 'taken\tVBN\n', 'from\tIN\n', 'several\tJJ\n', 'vantage\tNN\n']


34199

In [112]:
#corpus without tags, preprocessed
test_corpus = [get_word_tag(word, vocab)[0] for word in test_tag_corpus]# if word != '\n']
print(test_corpus[:10])

['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from', 'several', '--unk_noun--']


In [113]:
#corpus true tags
test_tags = [get_word_tag(word, vocab)[1] for word in test_tag_corpus]
print(test_tags[:10])

['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN', 'IN', 'JJ', 'NN']


In [12]:
len(test_corpus), len([w for w in test_corpus if w in vocab]), len(vocab)

(34199, 34199, 23777)

In [13]:
test_corpus[100:120]

['%',
 'gain',
 'reported',
 'Friday',
 'in',
 'the',
 'producer',
 'price',
 'index',
 '.',
 '--n--',
 'That',
 'gain',
 'was',
 'being',
 'cited',
 'as',
 'a',
 'reason',
 'the']

<a name='1'></a>
# Part 1: Parts-of-speech tagging 

<a name='1.1'></a>
## Part 1.1 - Training
We will start with the simplest possible parts-of-speech tagger and we will build up to the state of the art. 

In this section, We will find the words that are not ambiguous. 
- For example, the word `is` is a verb and it is not ambiguous. 
- In the `WSJ` corpus, $86$% of the token are unambiguous (meaning they have only one tag) 
- About $14\%$ are ambiguous (meaning that they have more than one tag)

<img src = "pos.png" style="width:400px;height:250px;"/>

Before we start predicting the tags of each word, we will need to compute a few dictionaries that will help generate the tables. 

#### Transition counts
- The first dictionary is the `transition_counts` dictionary which computes the number of times each tag happened next to another tag. 

This dictionary will be used to compute: 
$$P(t_i |t_{i-1}) \tag{1}$$

This is the probability of a tag at position $i$ given the tag at position $i-1$.

In order to compute equation 1, lets create a `transition_counts` dictionary where 
- The keys are `(prev_tag, tag)`
- The values are the number of times those two tags appeared in that order. 

#### Emission counts

The second dictionary  we will compute is the `emission_counts` dictionary. This dictionary will be used to compute:

$$P(w_i|t_i)\tag{2}$$

In other words, we will use it to compute the probability of a word given its tag. 

In order to compute equation 2, we will create an `emission_counts` dictionary where 
- The keys are `(tag, word)` 
- The values are the number of times that pair showed up in the training set. 

#### Tag counts

The last dictionary to compute is the `tag_counts` dictionary. 
- The key is the tag 
- The value is the number of times each tag appeared.

<a name='ex-01'></a>
### Exercise 01

Write a program that takes in the `training_corpus` and returns the three dictionaries mentioned above `transition_counts`, `emission_counts`, and `tag_counts`. 
- `emission_counts`: maps (tag, word) to the number of times it happened. 
- `transition_counts`: maps (prev_tag, tag) to the number of times it has appeared. 
- `tag_counts`: maps (tag) to the number of times it has occured. 


In [15]:
training_corpus[54]

'.\t.\n'

In [16]:
'\n'.split('\t')

['\n']

In [17]:
def create_dictionaries(training_corpus, vocab):
    """
    Input: 
        training_corpus: a corpus where each line has a word followed by its tag.
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output: 
        emission_counts: a dictionary where the keys are (tag, word) and the values are the counts
        transition_counts: a dictionary where the keys are (prev_tag, tag) and the values are the counts
        tag_counts: a dictionary where the keys are the tags and the values are the counts
    """
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    prev_tag = '--s--'
    
    for line in training_corpus:
        word, tag = get_word_tag(line, vocab)
        transition_counts[(prev_tag, tag)] += 1
        emission_counts[(tag, word)] += 1
        tag_counts[tag] += 1

        prev_tag = tag
        
    return emission_counts, transition_counts, tag_counts

In [18]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

In [19]:
# get all the POS states
print(f"Number of POS tags (number of 'states'): {len(tag_counts)}")
print([tag for tag in tag_counts])

Number of POS tags (number of 'states'): 46
['IN', 'DT', 'NNP', 'CD', 'NN', '``', "''", 'POS', '(', 'VBN', 'NNS', 'VBP', ',', 'CC', ')', 'VBD', 'RB', 'TO', '.', '--s--', 'VBZ', 'NNPS', 'PRP', 'PRP$', 'VB', 'JJ', 'MD', 'VBG', 'RBR', ':', 'WP', 'WDT', 'JJR', 'PDT', 'RBS', 'WRB', 'JJS', '$', 'RP', 'FW', 'EX', 'SYM', '#', 'LS', 'UH', 'WP$']


The 'states' are the Parts-of-speech designations found in the training data. They will also be referred to as 'tags' or POS in this assignment. 

- "NN" is noun, singular, 
- 'NNS' is noun, plural. 
- In addition, there are helpful tags like '--s--' which indicate a start of a sentence.
- A more complete description is at [Penn Treebank II tag set](https://www.clips.uantwerpen.be/pages/mbsp-tags). 

In [20]:
print("transition examples: ", tuple(transition_counts.items())[:3])
print("emission examples: ", tuple(emission_counts.items())[:3])
print("\nambigous word examples: ")
for tup,cnt in emission_counts.items():
    if tup[1] == 'back': print (tup, cnt) 

transition examples:  ((('--s--', 'IN'), 5050), (('IN', 'DT'), 32364), (('DT', 'NNP'), 9044))
emission examples:  ((('IN', 'In'), 1735), (('DT', 'an'), 3142), (('NNP', 'Oct.'), 317))

ambigous word examples: 
('RB', 'back') 304
('VB', 'back') 20
('RP', 'back') 84
('JJ', 'back') 25
('NN', 'back') 29
('VBP', 'back') 4


<a name='1.2'></a>
### Part 1.2 - Testing

Now we will test the accuracy of our parts-of-speech tagger using the `emission_counts` dictionary. 
- Given the preprocessed test corpus, we will assign a parts-of-speech tag to every word in that corpus. 
- Using the original tagged test corpus, we will then compute what percent of the tags we got correct. 

<a name='ex-02'></a>
### Exercise 02

Implement `predict_pos` that computes the accuracy of the model. 

In [118]:
def predict_pos(test_corpus, y, emission_counts, vocab, states):
    '''
    Input: 
        test_corpus: a preprocessed version of 'y'. A list with the 'word' component of the tuples.
        y: a corpus composed of a list of tuples where each tuple consists of (word, POS)
        emission_counts: a dictionary where the keys are (tag,word) tuples and the value is the count
        vocab: a dictionary where keys are words in vocabulary and value is an index
        states: a sorted list of all possible tags for this assignment
    Output: 
        accuracy: Number of times we classified a word correctly
    '''
    
    count = 0
    i = 0
    
    for word in test_corpus:
        if word not in vocab:
            i += 1
            continue
        
        candidates = {tag: emission_counts[(tag,word)] for tag in states 
                      if (tag,word) in emission_counts}
        pred_tag = [tag for tag in candidates if candidates[tag] == max(candidates.values())][0]
        
        # print(word, candidates, pred_tag, y[i][1])
        if pred_tag == y[i][1]:
            count += 1
        i += 1
        
    return round(count/i, 4)

In [22]:
states = sorted(tag_counts.keys())
print("Accuracy of prediction using predict_pos is", 
      predict_pos(test_corpus, [get_word_tag(line, vocab) for line in test_tag_corpus], 
                  emission_counts, vocab, states))

Accuracy of prediction using predict_pos is 0.9307


86.6% is really good for this warm up exercise. With hidden markov models, we should be able to get **95% accuracy.**

<a name='2'></a>
# Part 2: Hidden Markov Models for POS

Now we will build something more context specific. Concretely, we will be implementing a Hidden Markov Model (HMM) with a Viterbi decoder
- The HMM is one of the most commonly used algorithms in Natural Language Processing, and is a foundation to many deep learning techniques. 
- In addition to parts-of-speech tagging, HMM is used in speech recognition, speech synthesis, etc. 
- By completing this part of the assignment we will get a 95% accuracy on the same dataset we used in Part 1.

The Markov Model contains a number of states and the probability of transition between those states. 
- In this case, the states are the parts-of-speech. 
- A Markov Model utilizes a transition matrix, `A`. 
- A Hidden Markov Model adds an observation or emission matrix `B` which describes the probability of a visible observation when we are in a particular state. 
- In this case, the emissions are the words in the corpus
- The state, which is hidden, is the POS tag of that word.

<a name='2.1'></a>
## Part 2.1 Generating Matrices

### Creating the 'A' transition probabilities matrix
Now that we have `emission_counts`, `transition_counts`, and `tag_counts`, we will start implementing the Hidden Markov Model. 

This will allow us to quickly construct the 
- `A` transition probabilities matrix.
- and the `B` emission probabilities matrix. 

We will also use some smoothing when computing these matrices. 

Here is an example of what the `A` transition matrix would look like (it is simplified to 5 tags for viewing. It is 46x46 in this assignment.):


|**A**  |...|         RBS  |          RP  |         SYM  |      TO  |          UH|...
| --- ||---:-------------| ------------ | ------------ | -------- | ---------- |----
|**RBS**  |...|2.217069e-06  |2.217069e-06  |2.217069e-06  |0.008870  |2.217069e-06|...
|**RP**   |...|3.756509e-07  |7.516775e-04  |3.756509e-07  |0.051089  |3.756509e-07|...
|**SYM**  |...|1.722772e-05  |1.722772e-05  |1.722772e-05  |0.000017  |1.722772e-05|...
|**TO**   |...|4.477336e-05  |4.472863e-08  |4.472863e-08  |0.000090  |4.477336e-05|...
|**UH**  |...|1.030439e-05  |1.030439e-05  |1.030439e-05  |0.061837  |3.092348e-02|...
| ... |...| ...          | ...          | ...          | ...      | ...        | ...

Note that the matrix above was computed with smoothing. 

Each cell gives us the probability to go from one part of speech to another. 
- In other words, there is a 4.47e-8 chance of going from parts-of-speech `TO` to `RP`. 
- The sum of each row has to equal 1, because we assume that the next POS tag must be one of the available columns in the table.

The smoothing was done as follows: 

$$ P(t_i | t_{i-1}) = \frac{C(t_{i-1}, t_{i}) + \alpha }{C(t_{i-1}) +\alpha * N}\tag{3}$$

- $N$ is the total number of tags
- $C(t_{i-1}, t_{i})$ is the count of the tuple (previous POS, current POS) in `transition_counts` dictionary.
- $C(t_{i-1})$ is the count of the previous POS in the `tag_counts` dictionary.
- $\alpha$ is a smoothing parameter.

<a name='ex-03'></a>
### Exercise 03

Implement the `create_transition_matrix` below for all tags. The task is to output a matrix that computes equation 3 for each cell in matrix `A`. 

In [23]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    ''' 
    Input: 
        alpha: number used for smoothing
        tag_counts: a dictionary mapping each tag to its respective count
        transition_counts: transition count for the previous word and tag
    Output:
        A: matrix of dimension (num_tags,num_tags)
    '''
    N = len(tag_counts)
    A = np.zeros((N, N))
    
    for i,tag_i in enumerate(sorted(tag_counts.keys())):
        for j,tag_j in enumerate(sorted(tag_counts.keys())):
            A[i, j] = (alpha + transition_counts[(tag_i, tag_j)]) / (tag_counts[tag_i] + alpha*N)
            
    return A

In [24]:
A = create_transition_matrix(0.001, tag_counts, transition_counts)
print("A at row 0, col 0: ", A[0,0])
print("A at row 3, col 1: ", A[3,1])

print("Viewing a subset of transition matrix A")
pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )

A at row 0, col 0:  7.039972966503809e-06
A at row 3, col 1:  0.16910191896905374
Viewing a subset of transition matrix A


,RBS,RP,SYM,TO,UH
RBS,2.217069e-06,2.217069e-06,2.217069e-06,0.008870,2.217069e-06
RP,3.756509e-07,7.516775e-04,3.756509e-07,0.051089,3.756509e-07
SYM,1.722772e-05,1.722772e-05,1.722772e-05,0.000017,1.722772e-05
TO,4.477336e-05,4.472863e-08,4.472863e-08,0.000090,4.477336e-05
UH,1.030439e-05,1.030439e-05,1.030439e-05,0.061837,3.092348e-02


### Create the 'B' emission probabilities matrix

Now We will create the `B` transition matrix which computes the emission probability. 

We will use smoothing as defined below: 

$$P(w_i | t_i) = \frac{C(t_i, word_i)+ \alpha}{C(t_{i}) +\alpha * N}\tag{4}$$

- $C(t_i, word_i)$ is the number of times $word_i$ was associated with $tag_i$ in the training data (stored in `emission_counts` dictionary).
- $C(t_i)$ is the number of times $tag_i$ was in the training data (stored in `tag_counts` dictionary).
- $N$ is the number of words in the vocabulary
- $\alpha$ is a smoothing parameter. 

The matrix `B` is of dimension (num_tags, N), where num_tags is the number of possible parts-of-speech tags. 

Here is an example of the matrix, only a subset of tags and words are shown: 
<p style='text-align: center;'> <b>B Emissions Probability Matrix (subset)</b>  </p>

|**B**| ...|          725 |     adroitly |    engineers |     promoted |      synergy| ...|
|----|----|--------------|--------------|--------------|--------------|-------------|----|
|**CD**  | ...| **8.201296e-05** | 2.732854e-08 | 2.732854e-08 | 2.732854e-08 | 2.732854e-08| ...|
|**NN**  | ...| 7.521128e-09 | 7.521128e-09 | 7.521128e-09 | 7.521128e-09 | **2.257091e-05**| ...|
|**NNS** | ...| 1.670013e-08 | 1.670013e-08 |**4.676203e-04** | 1.670013e-08 | 1.670013e-08| ...|
|**VB**  | ...| 3.779036e-08 | 3.779036e-08 | 3.779036e-08 | 3.779036e-08 | 3.779036e-08| ...|
|**RB**  | ...| 3.226454e-08 | **6.456135e-05** | 3.226454e-08 | 3.226454e-08 | 3.226454e-08| ...|
|**RP**  | ...| 3.723317e-07 | 3.723317e-07 | 3.723317e-07 | **3.723317e-07** | 3.723317e-07| ...|
| ...    | ...|     ...      |     ...      |     ...      |     ...      |     ...      | ...|



<a name='ex-04'></a>
### Exercise 04
Implement the `create_emission_matrix` below that computes the `B` emission probabilities matrix. The task is to output a matrix that computes equation 4 for each cell in matrix `B`. 

In [25]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    '''
    Input: 
        alpha: tuning parameter used in smoothing 
        tag_counts: a dictionary mapping each tag to its respective count
        emission_counts: a dictionary where the keys are (tag, word) and the values are the counts
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output:
        B: a matrix of dimension (num_tags, len(vocab))
    '''
    N = len(tag_counts)
    B = np.zeros((N, len(vocab)))
    
    for i,tag in enumerate(sorted(tag_counts.keys())):
        for j,word in enumerate(vocab):
            B[i, j] = (alpha + emission_counts[(tag, word)]) / (tag_counts[tag] + alpha*N)
            
    return B

In [26]:
#B[rows, [vocab[c] for c in cols]].shape
#print(np.ix_([states.index(t) for t in rows],[vocab[c] for c in cols]))
#[np.reshape([states.index(t) for t in rows], (-1,1)), [vocab[c] for c in cols]]

In [27]:
# creating emission probability matrix. this takes a few minutes to run. 
B = create_emission_matrix(0.001, tag_counts, emission_counts, vocab)

# Try viewing emissions for a few words in a sample dataframe
print("B at row 0, col 0: ", B[0,0])
print("B at row 3, col 1: ", B[3,1])

# Choose POS tags to show in a sample dataframe and get their indices
rows = ['CD', 'NN', 'NNS', 'VB', 'RB', 'RP']

# Select words to display
cols = ['725', 'adroitly', 'engineers', 'promoted', 'synergy']

# Get the emissions for the sample of words, and the sample of POS tags
print("Viewing a subset of emission matrix B")
pd.DataFrame(B[[states.index(t) for t in rows]][:, [vocab[c] for c in cols]], index=rows, columns=cols)

B at row 0, col 0:  7.039972966503809e-06
B at row 3, col 1:  7.320397702566385e-07
Viewing a subset of emission matrix B


,725,adroitly,engineers,promoted,synergy
CD,8.206618e-05,2.734628e-08,2.734628e-08,2.734628e-08,2.734628e-08
NN,7.522471e-09,7.522471e-09,7.522471e-09,7.522471e-09,2.257493e-05
NNS,1.670675e-08,1.670675e-08,4.678057e-04,1.670675e-08,1.670675e-08
VB,3.782428e-08,3.782428e-08,3.782428e-08,3.782428e-08,3.782428e-08
RB,3.228926e-08,6.461082e-05,3.228926e-08,3.228926e-08,3.228926e-08
RP,3.756509e-07,3.756509e-07,3.756509e-07,3.756509e-07,3.756509e-07


<a name='3'></a>
# Part 3: Viterbi Algorithm and Dynamic Programming

In this part of the assignment we will implement the Viterbi algorithm which makes use of dynamic programming. Specifically, we will use two matrices, `A` and `B` to compute the Viterbi algorithm. We have decomposed this process into three main steps.

* **Initialization** - In this part we initialize the `best_paths` and `best_probabilities` matrices that we will be populating in `feed_forward`.
* **Feed forward** - At each step, we calculate the probability of each path happening and the best paths up to that point. 
* **Feed backward**: This allows us to find the best path with the highest probabilities. 

<a name='3.1'></a>
## Part 3.1:  Initialization 

we will start by initializing two matrices of the same dimension. 

- best_probs: Each cell contains the probability of going from one POS tag to a word in the corpus.

- best_paths: A matrix that helps trace through the best possible path in the corpus. 

<a name='ex-05'></a>
### Exercise 05
Task: To initialize the `best_probs` and the `best_paths` matrix. 

Both matrices will be initialized to zero except for column zero of `best_probs`.  
- Column zero of `best_probs` is initialized with the assumption that the first word of the corpus was preceded by a start token ("--s--"). 
- This allows us to reference the **A** matrix for the transition probability

Here is how to initialize column 0 of `best_probs`:
- The probability of the best path going from the start index to a given POS tag indexed by integer $i$ is denoted by $\textrm{best_probs}[s_{idx}, i]$.
- This is estimated as the probability that the start tag transitions to the POS denoted by index $i$: $\mathbf{A}[s_{idx}, i]$ AND that the POS tag denoted by $i$ emits the first word of the given corpus, which is $\mathbf{B}[i, vocab[corpus[0]]]$.
- Note that vocab[corpus[0]] refers to the first word of the corpus (the word at position 0 of the corpus). 
- **vocab** is a dictionary that returns the unique integer that refers to that particular word.

Conceptually, it looks like this:
$\textrm{best_probs}[s_{idx}, i] = \mathbf{A}[s_{idx}, i] \times \mathbf{B}[i, corpus[0] ]$


In order to avoid multiplying and storing small values on the computer, we'll take the log of the product, which becomes the sum of two logs:

$best\_probs[i,0] = log(A[s_{idx}, i]) + log(B[i, vocab[corpus[0]]$

Also, to avoid taking the log of 0 (which is defined as negative infinity), the code itself will just set $best\_probs[i,0] = float('-inf')$ when $A[s_{idx}, i] == 0$


So the implementation to initialize $best\_probs$ looks like this:

$ if A[s_{idx}, i] <> 0 : best\_probs[i,0] = log(A[s_{idx}, i]) + log(B[i, vocab[corpus[0]]$

$ if A[s_{idx}, i] == 0 : best\_probs[i,0] = float('-inf')$

The example below shows the initialization assuming the corpus starts with the phrase "Loss tracks upward".

<img src = "Initialize4.PNG"/>

We will represent infinity and negative infinity like this:

```CPP
float('inf')
float('-inf')
```

In [28]:
def initialize(states, tag_counts, A, B, corpus, vocab):
    '''
    Input: 
        states: a list of all possible parts-of-speech
        tag_counts: a dictionary mapping each tag to its respective count
        A: Transition Matrix of dimension (num_tags, num_tags)
        B: Emission Matrix of dimension (num_tags, len(vocab))
        corpus: a sequence of words whose POS is to be identified in a list 
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output:
        best_probs: matrix of dimension (num_tags, len(corpus)) of floats
        best_paths: matrix of dimension (num_tags, len(corpus)) of integers
    '''
    best_probs = np.zeros((len(tag_counts), len(corpus)))
    best_paths = np.zeros((len(tag_counts), len(corpus)))
    sidx = states.index("--s--")
    
    for i in range(len(tag_counts)):
        if A[sidx, i] != 0:
            best_probs[i, 0] = math.log(A[sidx, i]) + math.log(B[i, vocab[corpus[0]]])
        else:
            best_probs[i, 0] = float('-inf')
        
    return best_probs, best_paths

In [119]:
# Test the function
best_probs, best_paths = initialize(states, tag_counts, A, B, test_corpus, vocab)
print("best_probs[0,0]: ", round(best_probs[0,0], 2))
print("best_paths[2,3]: ", best_paths[2,3])

best_probs[0,0]:  -22.46
best_paths[2,3]:  0.0


<a name='3.2'></a>
## Part 3.2 Viterbi Forward

In this part of the assignment, we will implement the `viterbi_forward` segment. In other words, we will populate the `best_probs` and `best_paths` matrices.
- Walk forward through the corpus.
- For each word, compute a probability for each possible tag. 
- Unlike the previous algorithm `predict_pos` (the 'warm-up' exercise), this will include the path up to that (word,tag) combination. 

Here is an example with a three-word corpus "Loss tracks upward":
- Note, in this example, only a subset of states (POS tags) are shown in the diagram below, for easier reading. 
- In the diagram below, the first word "Loss" is already initialized. 
- The algorithm will compute a probability for each of the potential tags in the second and future words. 

Compute the probability that the tag of the second work ('tracks') is a verb, 3rd person singular present (VBZ).  
- In the `best_probs` matrix, go to the column of the second word ('tracks'), and row 40 (VBZ), this cell is highlighted in light orange in the diagram below.
- Examine each of the paths from the tags of the first word ('Loss') and choose the most likely path.  
- An example of the calculation for **one** of those paths is the path from ('Loss', NN) to ('tracks', VBZ).
- The log of the probability of the path up to and including the first word 'Loss' having POS tag NN is $-14.32$.  The `best_probs` matrix contains this value -14.32 in the column for 'Loss' and row for 'NN'.
- Find the probability that NN transitions to VBZ.  To find this probability, go to the `A` transition matrix, and go to the row for 'NN' and the column for 'VBZ'.  The value is $4.37e-02$, which is circled in the diagram, so add $-14.32 + log(4.37e-02)$. 
- Find the log of the probability that the tag VBS would 'emit' the word 'tracks'.  To find this, look at the 'B' emission matrix in row 'VBZ' and the column for the word 'tracks'.  The value $4.61e-04$ is circled in the diagram below.  So add $-14.32 + log(4.37e-02) + log(4.61e-04)$.
- The sum of $-14.32 + log(4.37e-02) + log(4.61e-04)$ is $-25.13$. Store $-25.13$ in the `best_probs` matrix at row 'VBZ' and column 'tracks' (as seen in the cell that is highlighted in light orange in the diagram).
- All other paths in best_probs are calculated.  Notice that $-25.13$ is greater than all of the other values in column 'tracks' of matrix `best_probs`, and so the most likely path to 'VBZ' is from 'NN'.  'NN' is in row 20 of the `best_probs` matrix, so $20$ is the most likely path.
- Store the most likely path $20$ in the `best_paths` table.  This is highlighted in light orange in the diagram below.

So the formula to compute the probability and path for the $i^{th}$ word in the $corpus$, the prior word $i-1$ in the corpus, current POS tag $j$, and previous POS tag $k$ is:

$\mathrm{prob} = \mathbf{best\_prob}_{k, i-1} + \mathrm{log}(\mathbf{A}_{k, j}) + \mathrm{log}(\mathbf{B}_{j, vocab(corpus_{i})})$

where $corpus_{i}$ is the word in the corpus at index $i$, and $vocab$ is the dictionary that gets the unique integer that represents a given word.

$\mathrm{path} = k$

where $k$ is the integer representing the previous POS tag.


<a name='ex-06'></a>

### Exercise 06

Lets implement the `viterbi_forward` algorithm and store the best_path and best_prob for every possible tag for each word in the matrices `best_probs` and `best_tags`.

<img src = "Forward4.PNG"/>

In [30]:
B.shape, len(test_corpus), best_probs.shape

((46, 23777), 34199, (46, 34199))

In [1]:
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):
    '''
    Input: 
        A, B: The transiton and emission matrices respectively
        test_corpus: a list containing a preprocessed corpus
        best_probs: an initilized matrix of dimension (num_tags, len(corpus))
        best_paths: an initilized matrix of dimension (num_tags, len(corpus))
        vocab: a dictionary where keys are words in vocabulary and value is an index 
    Output: 
        best_probs: a completed matrix of dimension (num_tags, len(corpus))
        best_paths: a completed matrix of dimension (num_tags, len(corpus))
    '''
    prob = np.zeros(best_probs.shape[0])
    
    for i in range(1, best_probs.shape[1]):
        for j in range(best_probs.shape[0]):
            for k in range(best_probs.shape[0]):
                prob[k] = best_probs[k, i-1] + math.log(A[k,j]) + math.log(B[j,vocab[test_corpus[i]]])
            
            k = prob.argmax()
            best_paths[j,i] = k
            best_probs[j,i] = prob[k] 
            # if (i % 5000 == 0) & (j % 10 == 0):
            #    print(k, B[j, vocab[test_corpus[i]]], prob)
            
        if i % 5000 == 0:
            print("Words processed: ", i)
    
    return best_probs, best_paths

In [85]:
B.shape, A.shape

((46, 23777), (46, 46))

Lets run the `viterbi_forward` function to fill in the `best_probs` and `best_paths` matrices.

**Note** that this will take a few minutes to run.  There are about 30,000 words to process.

In [122]:
# this will take a few minutes to run => processes ~ 30,000 words
best_probs, best_paths = viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab)

Words processed:  5000
Words processed:  10000
Words processed:  15000
Words processed:  20000
Words processed:  25000
Words processed:  30000


In [127]:
# Testing this function 
print(f"best_probs[0,1]: {best_probs[0,1]:.2f} and best_path = {best_paths[0,1]:.2f}")
print(f"best_probs[0,4]: {best_probs[0,4]:.2f} and best_path = {best_paths[0,4]:.2f}")

best_probs[0,1]: -24.63 and best_path = 11.00
best_probs[0,4]: -49.40 and best_path = 20.00


<a name='3.3'></a>
## Part 3.3 Viterbi backward

Now we will implement the Viterbi backward algorithm.
- The Viterbi backward algorithm gets the predictions of the POS tags for each word in the corpus using the `best_paths` and the `best_probs` matrices.

The example below shows how to walk backwards through the best_paths matrix to get the POS tags of each word in the corpus. Recall that this example corpus has three words: "Loss tracks upward".

POS tag for 'upward' is `RB`
- Select the the most likely POS tag for the last word in the corpus, 'upward' in the `best_prob` table.
- Look for the row in the column for 'upward' that has the largest probability.
- Notice that in row 28 of `best_probs`, the estimated probability is -34.99, which is larger than the other values in the column.  So the most likely POS tag for 'upward' is `RB` an adverb, at row 28 of `best_prob`. 
- The variable `z` is an array that stores the unique integer ID of the predicted POS tags for each word in the corpus.  In array z, at position 2, store the value 28 to indicate that the word 'upward' (at index 2 in the corpus), most likely has the POS tag associated with unique ID 28 (which is `RB`).
- The variable `pred` contains the POS tags in string form.  So `pred` at index 2 stores the string `RB`.


POS tag for 'tracks' is `VBZ`
- The next step is to go backward one word in the corpus ('tracks').  Since the most likely POS tag for 'upward' is `RB`, which is uniquely identified by integer ID 28, go to the `best_paths` matrix in column 2, row 28.  The value stored in `best_paths`, column 2, row 28 indicates the unique ID of the POS tag of the previous word.  In this case, the value stored here is 40, which is the unique ID for POS tag `VBZ` (verb, 3rd person singular present).
- So the previous word at index 1 of the corpus ('tracks'), most likely has the POS tag with unique ID 40, which is `VBZ`.
- In array `z`, store the value 40 at position 1, and for array `pred`, store the string `VBZ` to indicate that the word 'tracks' most likely has POS tag `VBZ`.

POS tag for 'Loss' is `NN`
- In `best_paths` at column 1, the unique ID stored at row 40 is 20.  20 is the unique ID for POS tag `NN`.
- In array `z` at position 0, store 20.  In array `pred` at position 0, store `NN`.

<img src = "Backwards5.PNG"/>

<a name='ex-07'></a>
### Exercise 07
Lets implement the `viterbi_backward` algorithm, which returns a list of predicted POS tags for each word in the corpus.

In [42]:
np.array([[10, 0, 0], 
          [330., 0., 50.], 
          [-30., 0., -40.], 
          [0., 5., 54.], 
          [2.2, 6, -4], 
          [2,-50, 0]]).argmax(0)

array([1, 4, 3], dtype=int64)

In [125]:
def viterbi_backward(best_probs, best_paths, corpus, states):
    '''
    This function returns the best path lists (both indices and tag names). 
    
    '''
    z = np.zeros(len(test_corpus), dtype=int)  # best_probs.argmax(0)
    pred = np.zeros(len(test_corpus), dtype='U5')  # [states[s] for s in z]
    
    z[-1] = best_probs[:, -1].argmax()
    pred[-1] = states[z[-1]]
    
    for i in range(len(test_corpus)-1, 0, -1):
        z[i-1] = best_paths[z[i], i]
        pred[i-1] = states[z[i-1]]
        
    return z, pred

In [126]:
# Testing the function for the last few words of the corpus and their states to aid in debug.
z, pred = viterbi_backward(best_probs, best_paths, test_corpus, states)
print("The prediction for pred[-7:m-1] is:\n", test_corpus[-7:-1], "\n", pred[-7:-1])
print("\nThe prediction for pred[0:8] is:\n", test_corpus[0:8], "\n", pred[0:8])

The prediction for pred[-7:m-1] is:
 ['see', 'them', 'here', 'with', 'us', '.'] 
 ['VB' 'PRP' 'RB' 'IN' 'PRP' '.']

The prediction for pred[0:8] is:
 ['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken', 'from'] 
 ['DT' 'NN' 'POS' 'NN' 'MD' 'VB' 'VBN' 'IN']


<a name='4'></a>
# Part 4: Predicting on a data set

Lets compute the accuracy of our prediction by comparing it with the true `y` labels. 
- `pred` is a list of predicted POS tags corresponding to the words of the `test_corpus`. 

In [116]:
print(f"The third word is: {test_corpus[3]}")
print("The prediction is: ", pred[3])

The third word is: temperature
The prediction is:  NN


<a name='ex-08'></a>
### Exercise 08

Lets compute the accuracy of the viterbi algorithm's POS tag predictions.

In [115]:
print("Accuracy of the Viterbi algorithm is {:.2f}".format(np.mean(pred==test_tags)))

Accuracy of the Viterbi algorithm is 0.96


As seen we were able to classify the parts-of-speech with 96% accuracy. 

### Key Points and overview

In this assignment we learned about parts-of-speech tagging. 
- In this assignment, we predicted POS tags by walking forward through a corpus and knowing the previous word.
- There are other implementations that use bidirectional POS tagging.
- Bidirectional POS tagging requires knowing the previous word and the next word in the corpus when predicting the current word's POS tag.
- Bidirectional POS tagging would tell more about the POS instead of just knowing the previous word. 
- Since we have learned to implement the unidirectional approach, we have the foundation to implement other POS taggers used in industry.

### References

- ["Speech and Language Processing", Dan Jurafsky and James H. Martin](https://web.stanford.edu/~jurafsky/slp3/)
- We would like to thank Melanie Tosik for her help and inspiration